<a href="https://colab.research.google.com/github/Jayzeferino/PDI/blob/main/TRabalhoPDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.util import img_as_ubyte
import math
from skimage.draw import disk

def butterworth(shape,d0,n):
  H=np.zeros(shape)
  for u in range(shape[0]):
    for v in range(shape[1]):
      D_uv=np.sqrt( (u-shape[0]/2)**2 + (v-shape[1]/2)**2 )
      H[u,v] = 1/ (1+ (D_uv/d0 )**(2*n))
  return H

img= io.imread('moon.png',as_gray=True)
f=np.fft.fft2(img)
f_s=np.fft.fftshift(f)


#Apenas para exibir
temp= np.log(abs(f_s))
io.imshow(temp, cmap="gray")

# #ideal passa baixa
# mask = np.zeros(img.shape,dtype=np.ubyte)
# rr,cc=disk((img.shape[0]/2,img.shape[1]/2),30,shape=img.shape)
# mask[rr,cc]=255
# filtered=f_s*temp
# filtered_unshift= np.fft.ifftshift(filtered)
# space_f= np.abs(np.fft.ifft2(filtered_unshift))
# io.imshow(temp, cmap="gray")







True